<a href="https://colab.research.google.com/github/tiagobnoronha/SCI-AI/blob/main/DesafioImersaoDEVAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação

Este projeto foi desenvolvido para a Imersão DEV Alura+Google, e tem como objetivo a criação de um sistema de checagem de informação baseado em AI.

## Como utilizar?

Para utilizar esse Notebook, é necessário criar uma chave secreta do Google Colab com o nome SECRET_API_KEY com o valor de sua API KEY do Google AI Studio.

Então, podem ser executadas todas as células do Notebook (Ctrl+F9).

Na última célula, será montada a interface gráfica do sistema de Checagem. É possível checar a informação nos seguintes formatos:

- Texto: se você possuir a informação no formato textual (e.g. mensagem em rede social, e-mail, etc.)
- URL: se você possui a URL da página que contém a informação que você quer validar.
- Imagem: se você recebeu a informação de forma textual, mas em uma imagem. O propósito dessa aba não é verificar se uma fotografia é verídica ou não, mas é voltada para imagens que possuam texto. São aceitos diversos formatos de imagem: jpg, png, bmp, gif, etc.
- Áudio: caso possua a informação em áudio. São aceitos diversos formatos de áudio: m4a, mp3, wav, etc.

Uma vez preenchido o formulário, basta clicar no botão **"Checar Fatos"**.

OBS. 1: Dependendo do tipo de informação, o tratamento e análise da informação pode demorar um pouco. As etapas intermediárias são exibidas no widget de Output. Aguarde a análise aparecer.

OBS. 2: O sistema pode apresentar uma análise equivocada para fatos muito recentes. Devido ao tempo exíguo para o desenvolvimento do Desafio o sistema carece de validação para verificar seu índice de acerto. Essa é uma versão beta do sistema que será ajustada a posteriori, sendo assim, utilize-se com cautela. Todavia, pode-se observar que este protótipo apresentou um índice de acertos razoável provando que o conceito pode ser explorado.

## Instalação da Biblioteca Google Generative AI

In [11]:
!pip install -q -U google-generativeai

## Importação das Bibliotecas


In [12]:
from google.colab import userdata
import google.generativeai as genai
import google.ai.generativelanguage as glm
import requests
import ipywidgets as widgets
from IPython.display import display, Markdown


# Funções para Tratamento da Informação

Esta seção apresenta funções para lidar com as informações do usuário (texto, url, imagem ou áudio). São elas:

- **extrair_texto_de_url**: esta função recebe uma string com a URL da pagina que deseja-se verificar a informação. Utiliza o Gemini para fazer o WebScraping da página, extraindo o texto principal. É necessário passar o modelo por argumento para a função.
- **extrai_texto_de_imagem**: o objetivo dessa função é receber uma imagem que contenha texto, e realizar o OCR da imagem. O OCR é realizado utilizando as ferramentas de AI do Google. Para isso, deve-se passar a imagem no formato **glm.Blob** para a função e o modelo Gemini para realizar a transcrição.
- **extrai_texto_de_audio**: o objetivo dessa função é receber um áudio e realizar a sua transcrição. A transcrição é realizada utilizando as ferramentas de AI do Google.
- **verifica_veracidade**: as três funções anteriores transformam seu conteúdo de entrada em texto. Portanto, independente da fonte de entrada da informação, esta função terá o objetivo de validar se a informação passada pelo usuário é verdadeira ou se tem indícios de notícia falsa.


In [13]:
################################################################################
def extrai_texto_de_url(model, url):
  # Verifica se a URL passada é válida
  http_resp = requests.get(url)

  if http_resp.status_code != 200:
    raise Exception('ERRO: Não foi possível obter os dados da URL informada.')

  prompt = f'Extraia o texto principal desta página {url}'
  response = model.generate_content(prompt)
  return response.text

################################################################################
def extrai_texto_de_imagem(model, img):
  prompt = 'Extraia o texto presente na imagem'
  response = model.generate_content([prompt, img])
  return response.text

################################################################################
def extrai_texto_de_audio(model, audio):

  prompt = "Transcreva o audio exatamente com as palavras do locutor."
  response = model.generate_content([audio, prompt])

  return response.text

################################################################################
def verifica_veracidade(model, texto):
  prompt = f'''Verifique a veracidade do texto a seguir:
               {texto}
               Explique como chegou a conclusão.
               '''

  response = model.generate_content(prompt)
  return response.text

# Funções para Criação da GUI

Nesta seção são definidas funções para a criação dos componentes responsáveis por gerenciar a interface gráfica.

## Helpers

In [14]:
def print_output(md):
  with analiseOutput:
    display(Markdown(md))

## Handlers


In [15]:
################################################################################
############## Handler para tratar o upload de imagem ##########################
################################################################################
def handler_image_change(ev):
  file = list(uploader_imagem.value.values())[0]
  filetype = file['metadata']['type']
  imageView.value = file['content']


################################################################################
############## Handler para tratar o clique no botão principal #################
################################################################################
def handler_checar_fatos(ev):

  btn_verificar.disabled = True

  analiseOutput.clear_output() # Limpa o widget de Output

  index = tb.selected_index    # Verifica o índice da tab que está selecionada

  ###### Tratamento da informação de entrada ######
  # Com base no título da tab selecionada decide
  # como tratar (dependendo se é texto, URL, imagem ou áudio)
  #
  # Ao final do match, o texto a ser verificado estará
  # armazenado na variável texto_para_validar
  match(tb.get_title(index)):

    case 'Texto':
      # Para entrada de texto não é necessário nenhum tratamento.
      # Basta obter o texto da textArea
      texto_para_validar = form_texto.children[1].value

    case 'URL':
      # Se a entrada for uma URL, é necessário obter a URL digitada,
      # e realizar o WebScraping da página utilizando os modelos de AI.

      # Se usuário não digitar o protocolo, assume-se o protocolo http
      url = form_url.children[1].value
      if not url.startswith('http'):
        url = f'http://{url}'

      # Busca o texto principal da página
      print_output('**Buscando texto da página...**')
      texto_para_validar = extrai_texto_de_url(model,url)

    case 'Imagem':
      # Se a entrada for uma imagem, é necessário realizar o OCR
      # para extrair as informações da imagem. Para isso, será
      # utilizado os modelos de AI.

      # Lê a imagem que foi realizado upload, e cria um Blob para
      # ser utilizado no OCR
      file = list(uploader_imagem.value.values())[0]
      img = glm.Blob(
          mime_type = file['metadata']['type'],
          data=file['content']
      )

      # Extrai o texto da imagem
      print_output('**Extraindo texto da imagem...**')
      texto_para_validar = extrai_texto_de_imagem(modelVision, img)

    case 'Áudio':
      # Se a entrada for um áudio, é necessário realizar a transcrição.
      # Para isso, será utilizado os modelos de AI.

      # Lê o audio que foi realizado upload, e cria um Blob para
      # ser utilizado na função de transcrição
      file = list(uploader_audio.value.values())[0]
      aud = glm.Blob(
          mime_type = file['metadata']['type'],
          data=file['content']
      )

      # Transcreve o áudio
      print_output('**Transcrevendo áudio...**')
      texto_para_validar = extrai_texto_de_audio(modelAudio, aud)

  # Neste ponto, independente da fonte da informação, ela já estará
  # convertida em texto. Portanto, basta chamar a função de verificação
  # da informação.
  print_output('**Analisando informação...**')
  resposta = verifica_veracidade(model, texto_para_validar)

  # Limpa o widget de Output e exibe a análise do modelo sobre a informação
  analiseOutput.clear_output()
  print_output(resposta)

  btn_verificar.disabled = False

## Builders

In [16]:
################################################################################
############# Constrói o formulário de Entrada Textual #########################
################################################################################
def build_form_texto():

  txtTexto = widgets.Textarea(
                        placeholder='Entre com o texto que deseja checar...',
                        layout=widgets.Layout(widht='100%'),
                        rows=8)

  panel = widgets.VBox(children=[
      widgets.HTML('<h2>Informação Textual</h2>'),
      txtTexto
  ])

  return panel

################################################################################
################### Constrói o formulário de URL ###############################
################################################################################
def build_form_url():

  txtURL = widgets.Text(
                placeholder='Entre com a URL da página que deseja checar...',
                description='URL'
              )

  panel = widgets.VBox(children=[
      widgets.HTML('<h2>Informação URL</h2>'),
      txtURL
  ])

  return panel

################################################################################
################# Constrói o formulário de Imagem ##############################
################################################################################
def build_form_imagem():

  file_upload = widgets.FileUpload(
      accept='image/*',
      multiple=False,
      description='Imagem'
  )
  file_upload.observe(handler_image_change, names='value')

  #with open("./img_placeholder.png", "rb") as file:
  #  image = file.read()

  imageView = widgets.Image(
                        #value=image,
                        width=300
                      )

  panel = widgets.VBox(children=[
            widgets.HTML('<h2>Imagem</h2>'),
            file_upload,
            imageView
          ])



  return panel, file_upload, imageView

################################################################################
################## Constrói o formulário de Áudio ##############################
################################################################################
def build_form_audio():

  file_upload = widgets.FileUpload(
      accept='audio/*',
      multiple=False,
      description='Áudio',
  )
  #file_upload.observe(handler_audio_change, names='value')
  panel = widgets.VBox(children=[
      widgets.HTML('<h2>Áudio</h2>'),
      file_upload
  ])

  return panel, file_upload

# Gemini AI

## Conexão Google AI Studio

Para utilizar esse Colab, acresente uma chave secreta do Google Colab com o nome **SECRET_API_KEY** com o valor de sua API KEY do Google AI Studio, e dê acesso ao notebook.

In [17]:
API_KEY = userdata.get("SECRET_API_KEY")
genai.configure(api_key=API_KEY)

## Parametrização do Model

In [18]:
################################################################################
################ Configura modelo para verificação da informação ###############
################################################################################
# Este modelo é utilizado para fazer a validação da informação, e extração da
# informação principal de uma página. Como as duas atividades não exigem
# criatividade do modelo, a temperatura foi configurada como 0.
generation_config = {
  "candidate_count": 1,
  "temperature": 0,
}

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config)

################################################################################
########### Configura modelo para verificação extrair texto de imagem ##########
################################################################################
# Este modelo é utilizado para fazer o OCR das imagens utilizadas como fonte de
# informação. Foi preciso a criação de um novo modelo, pois é nécessário a
# utilização do gemini-1.0-pro-vision para lidar com imagens.
generation_config_imagem = {
  "candidate_count": 1,
  "temperature": 0,
}

modelVision = genai.GenerativeModel(model_name="gemini-1.0-pro-vision-latest",
                              generation_config=generation_config_imagem)

################################################################################
################## Configura modelo para transcrição de Áudio ##################
################################################################################
# Este modelo é utilizado para fazer a transcrição de áudio. Foi preciso a
# criação de um novo modelo, pois é nécessário a utilização do
# gemini-1.5-pro para lidar com áudios.
generation_config_audio = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 1048576,
  "candidate_count": 1
}


# Foi necessário remover todos os filtros, pois como o objetivo do modelo é
# transcrever áudio que contenha conteúdo potencialmente falso, é possível
# que o áudio contenha conteúdo sensível, e para realizar a validação da
# informação é importante transcrevê-lo literalmente. Observa-se que a
# transcrição é utilizada internamente pelo sistema, não sendo exibida para o
# usuário.
safety_settings_audio = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

modelAudio = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config_audio,
                              safety_settings=safety_settings_audio)



# Construção da GUI

O objetivo desta etapa é montar a interface gráfica.

In [20]:

#Cabeçalho do Formulário Principal
header = widgets.HTML('<h1>Sistema de Checagem de Informação</h1>')

# Formulários
form_texto = build_form_texto()
form_url = build_form_url()
form_imagem, uploader_imagem, imageView = build_form_imagem()
form_audio, uploader_audio = build_form_audio()

# Botão para verificar as informações
btn_verificar = widgets.Button(description='Checar os fatos',
                      layout=widgets.Layout(width='100%', height='100%'),
                      button_style='info',
                      icon='check'
                      )
btn_verificar.on_click(handler_checar_fatos)

# Cria a tabBar
tb = widgets.Tab(
              [form_texto, form_url, form_imagem, form_audio],
              layout={'width': '200%'}
            )

titles = ['Texto', 'URL', 'Imagem', 'Áudio']
for i in range(len(titles)):
  tb.set_title(i, titles[i])

# Cria widget de Output
analiseOutput = widgets.Output(
                          layout={
                              'border': '1px solid black',
                              'width': '150%',
                              'padding':'10px'
                              }
                          )

# Monta a interface e exibe
app = widgets.VBox([
        header,
        widgets.HBox([
            tb,
            btn_verificar,
            analiseOutput
        ]),

])
display(app)
